Testing the plate detection model


In [3]:
import cv2
from ultralytics import YOLO

model = YOLO('C:\\Users\\Aman\\Desktop\\thesisproject\\model\\best (1).pt')  # Update path
img_path = 'C:\\Users\\Aman\\Desktop\\thesisproject\\extracted_frames2\\frame_1200.jpg'  # Update path
results = model.predict(img_path, conf=0.5, imgsz=640)

img = cv2.imread(img_path)
for result in results:
    for box in result.boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(img, f'Plate {box.conf.item():.2f}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

cv2.imwrite('output.jpg', img)
cv2.imshow('Result', img)
cv2.waitKey(0)
cv2.destroyAllWindows()


image 1/1 C:\Users\Aman\Desktop\thesisproject\extracted_frames2\frame_1200.jpg: 384x640 1 plate, 213.1ms
Speed: 0.0ms preprocess, 213.1ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)


preprocessed to increase the performance of the ocr

In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO
import easyocr

# Initialize EasyOCR reader
reader = easyocr.Reader(['en'], gpu=False)

# Load YOLOv8 model
model = YOLO('C:\\Users\\Aman\\Desktop\\thesisproject\\model\\best (1).pt')

# Load image
img_path = 'C:\\Users\\Aman\\Desktop\\thesisproject\\extracted_frames2\\frame_1200.jpg'
img = cv2.imread(img_path)
if img is None:
    print(f"Failed to load image: {img_path}")
    exit()

# Predict with YOLOv8
results = model.predict(img, conf=0.01, imgsz=640, iou=0.45)
print(f"Number of detected plates: {len(results[0].boxes)}")

# Process detections
plate_texts = []
for result in results:
    for box in result.boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        conf = box.conf.item()
        print(f"Drawing box: ({x1}, {y1}, {x2}, {y2}), Confidence: {conf:.2f}")
        # Crop plate
        plate_img = img[max(0, y1):y2, max(0, x1):x2]
        # Preprocess with Laplacian kernel
        gray = cv2.cvtColor(plate_img, cv2.COLOR_BGR2GRAY)
        upscaled = cv2.resize(gray, None, fx=2, fy=2, interpolation=cv2.INTER_CUBIC)
        laplacian_kernel = np.array([[-1, -1, -1], [-1, 9, -1], [-1, -1, -1]], dtype=np.float32)
        sharpened = cv2.filter2D(upscaled, -1, laplacian_kernel)
        # Apply EasyOCR
        ocr_results = reader.readtext(sharpened, allowlist='0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ', detail=1)
        text = ocr_results[0][1] if ocr_results else ""
        plate_texts.append(text)
        # Draw bounding box and text
        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(img, f'{text} ({conf:.2f})', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
# Save and display
cv2.imwrite('C:\\Users\\Aman\\Desktop\\thesisproject\\output_with_text.jpg', img)
try:
    cv2.imshow('Result', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
except cv2.error:
    print("Display failed. Check output_with_text.jpg for bounding boxes.")
print("Detected License Plates:", plate_texts)

: 